In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm 

In [3]:
# Define MDP parameters
actions = (0, 1)  # (0=do nothing, 1=maintenance)
xi = (15, 30, 50) # break threshold per type of component
S = [list(range(x + 1)) for x in xi]
C = tuple([[0, 1]] * x + [[math.inf, 5]] for x in xi)
gamma = 0.9  # Discount factor

In [ ]:
w = [np.zeros((x + 1) * 2) for x in xi]

In [ ]:
# Create feature vector \phi(s,a) for a given comp_type, state s, and action a
def feature_vector(comp_type, s, a):
    length = (xi[comp_type] + 1) * 2  # number of (state, action) combos
    phi = np.zeros(length)
    index = s * 2 + a # 
    phi[index] = 1.0
    return phi